In [1]:
from typing import Sequence, Tuple

import clingo

In [2]:
def solve(programs: Sequence[str],
          parts_in_order: Sequence[Sequence[Tuple[str, Sequence[clingo.Symbol]]]] = ((("base", ()),),),
          grounding_context=None,
          sep: str = ' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        #print("[Debug]:", "Adding", program)
        ctl.add("base", (), program)

    for parts in parts_in_order:
        #print("[Debug]:", "Grounding", parts)
        ctl.ground(parts, grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [3]:
commodities = """

commodity("EUR").
commodity("Lager").

"""
solve([commodities]);

Answer 1: { commodity("EUR") commodity("Lager") }
SAT 1


In [4]:
time = """

time(start).
time(0).
time(1).
time(a2).
time(b2).
time(a3).
time(b3).
time(end).
startTime(start).
endTime(end).

time_after(start, 0).
time_after(0, 1).
time_after(1, a2).
time_after(1, b2).
time_after(a2, a3).
time_after(b2, b3).
time_after(a3, end).
time_after(b3, end).

"""
solve([time]);

Answer 1: { endTime(end) startTime(start) time(0) time(1) time(a2) time(a3) time(b2) time(b3) time(end) time(start) time_after(0,1) time_after(1,a2) time_after(1,b2) time_after(a2,a3) time_after(a3,end) time_after(b2,b3) time_after(b3,end) time_after(start,0) }
SAT 1


In [5]:
accounts = """

account_commodity("Alice:Kassa", "EUR").
account_commodity("Alice:Lager", "Good").
account_commodity("Bob:Kassa", "EUR").
account_commodity("Bob:Lager", "Good").
account_commodity("Alice:Eigenkapital:EUR", "EUR").
account_commodity("Bob:Eigenkapital:Good", "Good").

"""
solve([accounts]);

Answer 1: { account_commodity("Alice:Eigenkapital:EUR","EUR") account_commodity("Alice:Kassa","EUR") account_commodity("Alice:Lager","Good") account_commodity("Bob:Eigenkapital:Good","Good") account_commodity("Bob:Kassa","EUR") account_commodity("Bob:Lager","Good") }
SAT 1


In [6]:
init = """

obs_at(account_debits_credits_commodity(Account, 0, 0, Commodity), ST) :-
  startTime(ST),
  account_commodity(Account, Commodity).

"""
solve([init]);

Answer 1: {  }
SAT 1


<block>:4:3-16: info: atom does not occur in any rule head:
  startTime(ST)

<block>:5:3-40: info: atom does not occur in any rule head:
  account_commodity(Account,Commodity)



In [7]:
transaction = """

obs_at(account_debitChange_commodity(DebitAccount, Change, Commodity), T) :-
  time(T),
  account_commodity(DebitAccount, Commodity),
  Change > 0,
  Change = #sum{ Amount,CreditAccount : occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount, CreditAccount, Amount, Commodity), T) }.

obs_at(account_creditChange_commodity(CreditAccount, Change, Commodity), T) :-
  time(T),
  account_commodity(CreditAccount, Commodity),
  Change > 0,
  Change = #sum{ Amount,DebitAccount : occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount, CreditAccount, Amount, Commodity), T) }.

"""
solve([transaction]);

Answer 1: {  }
SAT 1


<block>:4:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:5:3-45: info: atom does not occur in any rule head:
  account_commodity(DebitAccount,Commodity)

<block>:7:41-143: info: atom does not occur in any rule head:
  occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount,CreditAccount,Amount,Commodity),T)

<block>:10:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:11:3-46: info: atom does not occur in any rule head:
  account_commodity(CreditAccount,Commodity)

<block>:13:40-142: info: atom does not occur in any rule head:
  occ_at(debitAccount_creditAccount_amount_commodity(DebitAccount,CreditAccount,Amount,Commodity),T)



In [8]:
inertia = """

obs_at(account_debits_credits_commodity(Account, PreviousDebits + Debits, PreviousCredits + Credits, Commodity), NextTime) :-
  time_after(Time, NextTime),
  account_commodity(Account, Commodity),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), Time),
  obs_at(account_debitChange_commodity(Account, Debits, Commodity), Time),
  obs_at(account_creditChange_commodity(Account, Credits, Commodity), Time).

obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits + Credits, Commodity), NextTime) :-
  time_after(Time, NextTime),
  account_commodity(Account, Commodity),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), Time),
  not obs_at(account_debitChange_commodity(Account, _, Commodity), Time),
  obs_at(account_creditChange_commodity(Account, Credits, Commodity), Time).

obs_at(account_debits_credits_commodity(Account, PreviousDebits + Debits, PreviousCredits, Commodity), NextTime) :-
  time_after(Time, NextTime),
  account_commodity(Account, Commodity),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), Time),
  obs_at(account_debitChange_commodity(Account, Debits, Commodity), Time),
  not obs_at(account_creditChange_commodity(Account, _, Commodity), Time).

obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), NextTime) :-
  time_after(Time, NextTime),
  account_commodity(Account, Commodity),
  obs_at(account_debits_credits_commodity(Account, PreviousDebits, PreviousCredits, Commodity), Time),
  not obs_at(account_debitChange_commodity(Account, _, Commodity), Time),
  not obs_at(account_creditChange_commodity(Account, _, Commodity), Time).

"""
solve([inertia]);

Answer 1: {  }
SAT 1


<block>:4:3-29: info: atom does not occur in any rule head:
  time_after(Time,NextTime)

<block>:5:3-40: info: atom does not occur in any rule head:
  account_commodity(Account,Commodity)

<block>:7:3-74: info: atom does not occur in any rule head:
  obs_at(account_debitChange_commodity(Account,Debits,Commodity),Time)

<block>:8:3-76: info: atom does not occur in any rule head:
  obs_at(account_creditChange_commodity(Account,Credits,Commodity),Time)

<block>:11:3-29: info: atom does not occur in any rule head:
  time_after(Time,NextTime)

<block>:12:3-40: info: atom does not occur in any rule head:
  account_commodity(Account,Commodity)

<block>:14:7-73: info: atom does not occur in any rule head:
  obs_at(account_debitChange_commodity(#X0,#P1,#X2),#X3)

<block>:15:3-76: info: atom does not occur in any rule head:
  obs_at(account_creditChange_commodity(Account,Credits,Commodity),Time)

<block>:18:3-29: info: atom does not occur in any rule head:
  time_after(Time,NextTime)

<block>:19

In [9]:
instance = """
occ_at(debitAccount_creditAccount_amount_commodity("Alice:Kassa", "Alice:Eigenkapital:EUR", 100, "EUR"),0).
occ_at(debitAccount_creditAccount_amount_commodity("Bob:Kassa", "Alice:Kassa", 50, "EUR"), 1).
occ_at(debitAccount_creditAccount_amount_commodity("Alice:Lager", "Bob:Lager", 1, "Good"), a2).
occ_at(debitAccount_creditAccount_amount_commodity("Bob:Kassa", "Alice:Kassa", 50, "EUR"), a2).


"""
solve([instance]);

Answer 1: { occ_at(debitAccount_creditAccount_amount_commodity("Alice:Kassa","Alice:Eigenkapital:EUR",100,"EUR"),0) occ_at(debitAccount_creditAccount_amount_commodity("Alice:Lager","Bob:Lager",1,"Good"),a2) occ_at(debitAccount_creditAccount_amount_commodity("Bob:Kassa","Alice:Kassa",50,"EUR"),1) occ_at(debitAccount_creditAccount_amount_commodity("Bob:Kassa","Alice:Kassa",50,"EUR"),a2) }
SAT 1


In [10]:
solve([
    commodities,
    time,
    accounts,
    init,
    transaction,
    inertia,
    instance,
    "#show occ_at/2. #show obs_at/2."
], sep='\n');

Answer 1: {
obs_at(account_creditChange_commodity("Alice:Eigenkapital:EUR",100,"EUR"),0)
obs_at(account_creditChange_commodity("Alice:Kassa",50,"EUR"),1)
obs_at(account_creditChange_commodity("Alice:Kassa",50,"EUR"),a2)
obs_at(account_creditChange_commodity("Bob:Lager",1,"Good"),a2)
obs_at(account_debitChange_commodity("Alice:Kassa",100,"EUR"),0)
obs_at(account_debitChange_commodity("Alice:Lager",1,"Good"),a2)
obs_at(account_debitChange_commodity("Bob:Kassa",50,"EUR"),1)
obs_at(account_debitChange_commodity("Bob:Kassa",50,"EUR"),a2)
obs_at(account_debits_credits_commodity("Alice:Eigenkapital:EUR",0,0,"EUR"),0)
obs_at(account_debits_credits_commodity("Alice:Eigenkapital:EUR",0,0,"EUR"),start)
obs_at(account_debits_credits_commodity("Alice:Eigenkapital:EUR",0,100,"EUR"),1)
obs_at(account_debits_credits_commodity("Alice:Eigenkapital:EUR",0,100,"EUR"),a2)
obs_at(account_debits_credits_commodity("Alice:Eigenkapital:EUR",0,100,"EUR"),a3)
obs_at(account_debits_credits_commodity("Alice:Eigenka

Notice the two diverging ends:
- a-branch: Bob delivered the good. Alice paid the second half of the invoice.
- b-branch: Bob did not deliver the good. Alice did not pay.